In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import tensorflow_datasets as tfds

In [14]:
mnist_dataset, mnist_info = tfds.load('mnist',with_info = True, as_supervised=True)

In [57]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples =  int(0.1*mnist_info.splits['train'].num_examples)
num_test_samples = int(mnist_info.splits['test'].num_examples)

In [63]:
def scale(image,label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

In [66]:
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [67]:
buffer_size = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [68]:
batch_size = 100

train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [71]:
## Outline a model 

input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=[28,28,1]),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')])

In [74]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [76]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS,
         validation_data=(validation_inputs,validation_targets),
         validation_steps=1, verbose=2)

Epoch 1/5
540/540 - 21s - loss: 0.4046 - accuracy: 0.8865 - val_loss: 0.2188 - val_accuracy: 0.9347
Epoch 2/5
540/540 - 14s - loss: 0.1807 - accuracy: 0.9468 - val_loss: 0.1500 - val_accuracy: 0.9540
Epoch 3/5
540/540 - 12s - loss: 0.1359 - accuracy: 0.9598 - val_loss: 0.1310 - val_accuracy: 0.9567
Epoch 4/5
540/540 - 11s - loss: 0.1116 - accuracy: 0.9661 - val_loss: 0.1037 - val_accuracy: 0.9682
Epoch 5/5
540/540 - 14s - loss: 0.0933 - accuracy: 0.9711 - val_loss: 0.0947 - val_accuracy: 0.9690


In [77]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 4s 4s/step - loss: 0.1169 - accuracy: 0.9637

In [79]:
print("The loss is {0:.2f} and the accuracy is {1:.2f}".format(test_loss,test_accuracy*100))

The loss is 0.12 and the accuracy is 96.37
